In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import networkx as nx
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [2]:
import pickle
from scipy import sparse

In [3]:
with open('DBLP_Triplets.pickle','rb') as fh :
    data_obj = pickle.load(fh)

In [4]:
len(data_obj[0])

37791

In [5]:
data_obj[2][80000]

('P162545', 'T35', 'PT')

In [6]:
author_label = pd.read_csv('author_label.txt',low_memory=False,header=None,sep='\t',names=['ID','label','name'])

In [7]:
author_label

,ID,label,name
0,366357,1,Hoi-Yee Hwang
1,421581,3,Tee Kiah Chia
2,30266,0,Soumyadeb Mitra
3,136630,3,Azin Ashkan
4,114115,0,A. Kumaran
...,...,...,...
4052,19076,2,Bon K. Sy
4053,84215,1,Sergei O. Kuznetsov
4054,52975,3,David R. H. Miller
4055,19543,3,Richard M. Tong


In [10]:
Counter(author_label['label'])

Counter({1: 745, 3: 1006, 0: 1197, 2: 1109})

In [11]:
# Create the graph first

In [12]:
author_file = 'author.txt' 
author_df = pd.read_csv(
    author_file,
    low_memory=False,
    header=None,
    sep='\t',
    names=['ID','name']
)

In [13]:
paper_file = 'paper.txt' 
paper_df = pd.read_csv(
    paper_file,
    low_memory=False,
    header=None,
    sep='\t',
    names=['ID','name']
)

In [14]:
term_file = 'term.txt' 
term_df = pd.read_csv(
    term_file,
    low_memory=False,
    header=None,
    sep='\t',
    names=['ID','name']
)

In [15]:
conf_file = 'conf.txt' 
conf_df = pd.read_csv(
    conf_file,
    low_memory=False,
    header=None,
    sep='\t',
    names=['ID','name']
)

In [16]:
paper_ID_list = paper_df['ID'].values
author_ID_list = author_df['ID'].values
conf_ID_list = conf_df['ID'].values
term_ID_list = term_df['ID'].values

In [17]:
# --------------
# Make sure all type of nodes have IDs starting from 0
# --------------
class node:
    def __init__(_id,category):
        self.id = _id
        self.category = category
        

In [18]:
# Read in edges

PA = pd.read_csv('paper_author.txt', sep='\t', names = ['P','A'])
PC = pd.read_csv('paper_conf.txt', sep='\t', names = ['P','C'])
PT = pd.read_csv('paper_term.txt', sep='\t', names = ['P','T'])


In [19]:
nodes_dict = {
    'P': {},
    'A': {},
    'C': {},
    'T': {}
}


In [20]:
def search(df, val):
    return df.loc[df['ID']==val].values[0]


In [21]:
t_df = pd.read_csv(
    term_file,
    low_memory=False,
    header=None,
    sep='\t',
    names=['ID','name']
)

rmv_terms = t_df.loc[t_df['name'].isin(set(stopwords.words('english')))]['ID'].values.tolist()

In [22]:
# --------------------------
# Create graph 
# --------------------------
G = nx.Graph()
for i,row in PA.iterrows():
    p = row['P']
    a = row['A']
    if a in list(author_label['ID']):
        G.add_edge(p,a)
    
for i,row in PC.iterrows():
    p = row['P']
    c = row['C']
    G.add_edge(p,c)  
    
for i,row in PT.iterrows():
    p = row['P']
    t = row['T']
    if t not in rmv_terms:
        G.add_edge(p,t)

In [23]:
print(Counter(author_label['label']))

Counter({0: 1197, 2: 1109, 3: 1006, 1: 745})


In [24]:
label_0_ids =author_label.loc[author_label['label']==0]['ID'].values.tolist()
label_1_ids =author_label.loc[author_label['label']==1]['ID'].values.tolist()
label_2_ids =author_label.loc[author_label['label']==2]['ID'].values.tolist()
label_3_ids =author_label.loc[author_label['label']==3]['ID'].values.tolist()

In [26]:
G1 = G.copy()
print(G1.number_of_nodes(),G1.number_of_edges())

for conn_comp in nx.connected_components(G1):
    print(len(conn_comp))
G1.number_of_edges()
filter_terms = []
for node in G1.nodes():
    if node in list(term_df['ID']):
        if nx.degree(G1,node) < 3 : 
            filter_terms.append(node)
print(' Rare Terms to remove ',len(filter_terms))

for n in filter_terms:
    G1.remove_node(n)
isolates = [n for n in nx.isolates(G1) ]
for n in isolates:
    G1.remove_node(n)

print(G1.number_of_nodes(), G1.number_of_edges())

26126 122694
26126
 Rare Terms to remove  4670
21456 116983


In [27]:
# =================================
# Create synthetic ids for continuous ids in the riginal dataset
# ==================================
def get_df(id_list, valid_ids, label_df=None):
    id_list = [i for i in id_list if i in valid_ids]
    print(' >> ', len(id_list))
    data = [( i,j) for i,j in enumerate(id_list)]
    df = pd.DataFrame(np.array(data),columns=['synID','ID'])
    print(df.head(10))
    if label_df is not None:
         df = df.merge(label_df, on=['ID'],how='inner')
    return df

valid_ids = list(G1.nodes())

In [28]:
C_df = get_df(conf_ID_list,valid_ids)
T_df = get_df(term_ID_list,valid_ids)
P_df = get_df(paper_ID_list,valid_ids)

 >>  20
   synID    ID
0      0    36
1      1   597
2      2   755
3      3  1194
4      4  1201
5      5  1234
6      6  1798
7      7  1801
8      8  1902
9      9  2180
 >>  4144
   synID  ID
0      0   2
1      1   3
2      2   5
3      3   6
4      4   8
5      5   9
6      6  11
7      7  12
8      8  13
9      9  14
 >>  14376
   synID    ID
0      0  7601
1      1  7604
2      2  7605
3      3  7610
4      4  7612
5      5  7613
6      6  7621
7      7  7630
8      8  7631
9      9  7632


In [29]:
author_label_df = author_label[['ID','label']]
A_df = get_df(author_ID_list,valid_ids,author_label_df)

 >>  4321
   synID   ID
0      0   76
1      1  124
2      2  192
3      3  226
4      4  234
5      5  250
6      6  310
7      7  325
8      8  337
9      9  377


In [30]:
C_df['type'] = 'C'
P_df['type'] = 'P'
T_df['type'] = 'T'

C_df['label'] = None
P_df['label'] = None
T_df['label'] = None

In [31]:
A_df['type'] = 'A'

In [32]:
len(A_df)

4050

In [33]:
# Export the edge lists

In [34]:
G1.number_of_edges()

116983

In [43]:
all_nodes_df = A_df.copy()
all_nodes_df = all_nodes_df.append(C_df,ignore_index=True)
all_nodes_df = all_nodes_df.append(P_df,ignore_index=True)
all_nodes_df = all_nodes_df.append(T_df,ignore_index=True)


In [44]:
all_nodes_df['ID']

0          192
1          226
2          234
3          435
4          444
         ...  
22585    13401
22586    13440
22587    13451
22588    13485
22589    13548
Name: ID, Length: 22590, dtype: int64

In [63]:
edges_df = pd.DataFrame(columns=['n1','n2','edge_type'])


In [64]:
from joblib import Parallel, delayed

def setup_edges (e):
    row1 = all_nodes_df.loc[all_nodes_df['ID']==e[0]]
    row2 = all_nodes_df.loc[all_nodes_df['ID']==e[1]]
    
    id1 = row1['synID'].values.tolist()[0]
    id2 = row2['synID'].values.tolist()[0]
    type1 = row1['type'].values.tolist()[0]
    type2 = row2['type'].values.tolist()[0]
    
    # Lexicographic ordering
    if type1 > type2:
        type1, type2 = type2, type1
        row1, row2 = row2,row1
        id1, id2 = id2, id1

    
    edge_type = type1 + '_' + type2
    _dict = {
            'n1':id1,
            'n2':id2,
            'edge_type' : edge_type
        }
    return _dict
     
 

In [65]:
results = Parallel(n_jobs=16)(delayed(setup_edges)(e) for  e in G1.edges())

In [67]:
# for r in results:
#     edges_df = edges_df.append(r,ignore_index=True)

edges_df = pd.DataFrame(results)

In [68]:
edges_df.to_csv('dblp_edges.csv',index=False)

In [69]:
all_nodes_df.to_csv('dblp_nodes.csv',index=False)